In [1]:
import pandas as pd
from pathlib import Path

In [2]:
og_df = pd.read_csv(
    "/home/clay/research/kaggle/sennet/data/sennet_igor/data/gt_extended.csv",
    delimiter=";",
    index_col=0,
)

In [3]:
og_df.head()

,id,group,slice,height,width,img_path,msk_path
0,kidney_1_dense_0000,kidney_1_dense,0,1303,912,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...
1,kidney_1_dense_0001,kidney_1_dense,1,1303,912,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...
2,kidney_1_dense_0002,kidney_1_dense,2,1303,912,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...
3,kidney_1_dense_0003,kidney_1_dense,3,1303,912,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...
4,kidney_1_dense_0004,kidney_1_dense,4,1303,912,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...


In [4]:
blacklist = ["kidney_3_merged"]
items = sorted([
    p
    for p in Path("/home/clay/research/kaggle/sennet/data/blood-vessel-segmentation/train").glob("*")
    if p.is_dir
    and not any(b in p.name for b in blacklist)
])
items

[PosixPath('/home/clay/research/kaggle/sennet/data/blood-vessel-segmentation/train/kidney_1_dense'),
 PosixPath('/home/clay/research/kaggle/sennet/data/blood-vessel-segmentation/train/kidney_1_dense_xz'),
 PosixPath('/home/clay/research/kaggle/sennet/data/blood-vessel-segmentation/train/kidney_1_dense_zy'),
 PosixPath('/home/clay/research/kaggle/sennet/data/blood-vessel-segmentation/train/kidney_1_voi'),
 PosixPath('/home/clay/research/kaggle/sennet/data/blood-vessel-segmentation/train/kidney_1_voi_xz'),
 PosixPath('/home/clay/research/kaggle/sennet/data/blood-vessel-segmentation/train/kidney_1_voi_zy'),
 PosixPath('/home/clay/research/kaggle/sennet/data/blood-vessel-segmentation/train/kidney_2'),
 PosixPath('/home/clay/research/kaggle/sennet/data/blood-vessel-segmentation/train/kidney_2_xz'),
 PosixPath('/home/clay/research/kaggle/sennet/data/blood-vessel-segmentation/train/kidney_2_zy'),
 PosixPath('/home/clay/research/kaggle/sennet/data/blood-vessel-segmentation/train/kidney_3_dense

In [6]:
from tqdm import tqdm
import cv2


data_root = Path("/home/clay/research/kaggle/sennet/data")
new_data = {
    c: []
    for c in og_df.columns
}
print(new_data)

for p in tqdm(items):
    images = sorted(list((p / "images").glob("*")))
    labels = sorted(list((p / "images").glob("*")))
    for i, (img, lab) in enumerate(zip(images, labels, strict=True)):
        read_img = cv2.imread(str(img), 0)
        img_h, img_w = read_img.shape
        # print(img.relative_to(data_root))

        new_data['id'].append(f"{p.name}_{str(i).zfill(4)}")
        new_data['group'].append(p.name)
        new_data['slice'].append(i)
        new_data['height'].append(img_h)
        new_data['width'].append(img_w)
        new_data['img_path'].append(str(img.relative_to(data_root)))
        new_data['msk_path'].append(str(lab.relative_to(data_root)))
        # break

{'id': [], 'group': [], 'slice': [], 'height': [], 'width': [], 'img_path': [], 'msk_path': []}


100%|███████████████████████████████████████████████████████████████████| 18/18 [08:02<00:00, 26.81s/it]


In [8]:
new_df = pd.DataFrame(new_data)
new_df.head()

,id,group,slice,height,width,img_path,msk_path
0,kidney_1_dense_0000,kidney_1_dense,0,1303,912,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...
1,kidney_1_dense_0001,kidney_1_dense,1,1303,912,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...
2,kidney_1_dense_0002,kidney_1_dense,2,1303,912,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...
3,kidney_1_dense_0003,kidney_1_dense,3,1303,912,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...
4,kidney_1_dense_0004,kidney_1_dense,4,1303,912,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...


In [10]:
new_df.to_csv(
    "/home/clay/research/kaggle/sennet/data/sennet_igor/data/gt_extended_sumo.csv",
    sep=";",
    # index_col=0,
)